## Imports and API

In [2]:
# Libraries
import openai
import requests
import feedparser
from dotenv import load_dotenv
import os

load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("API_KEY")

## ArVix search agent

In [3]:
def search_arxiv(query, max_results=5):
    url = f'http://export.arxiv.org/api/query?search_query={query}&start=0&max_results={max_results}'
    feed = feedparser.parse(requests.get(url).text)
    
    results = []
    for entry in feed.entries:
        results.append({
            'title': entry.title,
            'summary': entry.summary,
            'link': entry.link
        })
    return results


## OpenAlex search agent

In [4]:
def search_openalex(query, max_results=5):
    url = f"https://api.openalex.org/works?search={query}&per-page={max_results}"
    response = requests.get(url)
    data = response.json()
    
    results = []
    for item in data.get('results', []):
        results.append({
            'title': item.get('display_name', 'No Title'),
            'summary': item.get('abstract_inverted_index', {}),
            'link': item.get('id', 'No Link')
        })
    return results


In [5]:
def merge_results(arxiv_results, openalex_results):
    return arxiv_results + openalex_results

## Summerizing agent

In [ ]:
def summarize_papers(papers):
    content = "\n\n".join([f"Title: {p['title']}\nSummary: {p['summary']}" for p in papers])
    prompt = f"Summarize the following research papers:\n{content}"
    
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    return response['choices'][0]['message']['content']